# Рубежный контроль №1
**"Монотонный итеративный алгоритм решения матричной игры"**

**Выполнил: Александров А. Н., ИУ8-104**

**Вариант: 1**

## Задание
Пусть с помощью матрицы А задана матричная антагонистическая игра двух игроков. По данной матрице необходимо найти решение игры для первого игрока (то есть, вектор оптимальных стратегий и цену игры), используя монотонный итеративный алгоритм.

Игра Г размера (3 x 3) задана матрицей:
 
| 1  | 11 | 11 |
|:--:|:--:|:--:|
| 7  | 5  | 8  | 
| 16 | 6  | 2  | 


In [1]:
import logging

import numpy as np

from game_theory.utils.matrix_games.monotonic.monotonic import Monotonic

from game_theory.utils.matrix_games.game_matrix import GameMatrix
from game_theory.utils.matrix_games.mixed_strategies import check_resulted_game_price, get_resulted_mixed_strategies

logging.basicConfig(level=logging.INFO, format='%(message)s')

In [2]:
# Входная матрица прямоугольной игры с нулевой суммой.
game_matrix = GameMatrix(
    matrix=np.array(
        [
            [1, 11, 11],
            [7, 5, 8],
            [16, 6, 2],
        ],
        dtype=int,
    )
)
game_matrix

+-----------------------------------------------+
|          Таблица стратегий (игрока А)         |
+----------------+----+----+----+---------------+
|   Стратегии    | b1 | b2 | b3 | MIN выигрыш A |
+----------------+----+----+----+---------------+
|       a1       | 1  | 11 | 11 |       1       |
|       a2       | 7  | 5  | 8  |       5       |
|       a3       | 16 | 6  | 2  |       2       |
| MAX проигрыш B | 16 | 11 | 11 |               |
+----------------+----+----+----+---------------+

In [3]:
print(f"Нижняя цена игры:  {game_matrix.lowest_game_price[1]}\n"
      f"Верхняя цена игры: {game_matrix.highest_game_price[1]}")

Нижняя цена игры:  5
Верхняя цена игры: 11


In [4]:
reduced_game: GameMatrix = game_matrix.reduce_dimension(method='dominant_absorption')
reduced_game

+-----------------------------------------------+
|          Таблица стратегий (игрока А)         |
+----------------+----+----+----+---------------+
|   Стратегии    | b1 | b2 | b3 | MIN выигрыш A |
+----------------+----+----+----+---------------+
|       a1       | 1  | 11 | 11 |       1       |
|       a2       | 7  | 5  | 8  |       5       |
|       a3       | 16 | 6  | 2  |       2       |
| MAX проигрыш B | 16 | 11 | 11 |               |
+----------------+----+----+----+---------------+

### Монотонный итеративный алгоритм

In [5]:
reduced_game

+-----------------------------------------------+
|          Таблица стратегий (игрока А)         |
+----------------+----+----+----+---------------+
|   Стратегии    | b1 | b2 | b3 | MIN выигрыш A |
+----------------+----+----+----+---------------+
|       a1       | 1  | 11 | 11 |       1       |
|       a2       | 7  | 5  | 8  |       5       |
|       a3       | 16 | 6  | 2  |       2       |
| MAX проигрыш B | 16 | 11 | 11 |               |
+----------------+----+----+----+---------------+

In [6]:
monotonic_method = Monotonic(reduced_game)

In [7]:
(
    (game_price_for_a, player_a_mixed_strategies),
    (game_price_for_b, player_b_mixed_strategies),
) = monotonic_method.solve()

Решение игры относительно игрока A
Итерация 0:
	x^0 = [0 0 1]
	c^0 = [16.  6.  2.]
	v^0 = 2.0
	J^0 = [3]

Итерация 1:
Рассмотрим подыгру Г^1: 
[[11.]
 [ 8.]
 [ 2.]]
Седловая точка найдена: (11.0, [1, 0, 0])
Оптимальная стратегия игрока: 
	‾x_1 = [1 0 0]
	‾c_1 = [ 1. 11. 11.]
Находим оптимальную стратегию игрока в подыгре из двух строк: 
[[16.  6.  2.]
 [ 1. 11. 11.]]
Удаление NBR-стратегий ['b2']
Матрица после уменьшения размерности: 
[[16.  2.]
 [ 1. 11.]]
F = b^T⋅x -> min,
A^T⋅x >= c^T,
x1, x2, ..., xn >= 0
b^T = [1 1]
A^T =
[[16.  1.]
 [ 2. 11.]],
c^T = [1 1].
Процесс решения:
Поиск опорного решения: 
Исходная симплекс-таблица:
+----+---------+----------+----------+
|    |   Si0   |    x1    |    x2    |
+----+---------+----------+----------+
| x3 | -1.0000 | -16.0000 | -1.0000  |
| x4 | -1.0000 | -2.0000  | -11.0000 |
| F  |  0.0000 | -1.0000  | -1.0000  |
+----+---------+----------+----------+
Разрешающая строка: x3
Разрешающий столбец: x1
+----+---------+---------+----------+
|  

In [8]:
# Смешанные стратегии игрока A и цена игры.
assert check_resulted_game_price(
    game_matrix=reduced_game,
    game_price_value=game_price_for_a,
)

mixed_strategies = get_resulted_mixed_strategies(
    player_labels=game_matrix.player_a_strategy_labels,
    labels_to_probability=dict(zip(
        reduced_game.player_a_strategy_labels,
        list(player_a_mixed_strategies),
    )),
    player_name="A",
)
print(mixed_strategies)

Цена игры: 5 <= 7.389 <= 11


+------------------------------+
| Смешанные стратегии игрока A |
+---------+---------+----------+
|    a1   |    a2   |    a3    |
+---------+---------+----------+
|  0.352  |  0.370  |  0.278   |
+---------+---------+----------+


In [9]:
# Смешанные стратегии игрока B и цена игры.
assert check_resulted_game_price(
    game_matrix=reduced_game,
    game_price_value=game_price_for_b,
)

mixed_strategies = get_resulted_mixed_strategies(
    player_labels=game_matrix.player_b_strategy_labels,
    labels_to_probability=dict(zip(
        reduced_game.player_b_strategy_labels,
        list(player_b_mixed_strategies),
    )),
    player_name="B",
)
print(mixed_strategies)

Цена игры: 5 <= 7.389 <= 11


+------------------------------+
| Смешанные стратегии игрока B |
+---------+---------+----------+
|    b1   |    b2   |    b3    |
+---------+---------+----------+
|  0.361  |  0.083  |  0.556   |
+---------+---------+----------+
